In [1]:
import psycopg2
import csv
import pandas as pd

In [2]:
def copy_to_postgres(filename, pg_cursor, table_name):
    with open(filename, 'r', encoding='utf-8-sig') as file:
        next(file)
        pg_cursor.copy_from(file, table_name, sep=',') 

In [3]:
########################### MUST CREATE CONNECTION to Postgres before any calls are made
pg_conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="darkdreamer17?",  # change this to your own password
        host="localhost",
        port="5432"
    )
pg_conn.autocommit = True
pg_cursor = pg_conn.cursor()


# 311 service requests for vacant/abandoned buildings
create_311_table = '''
    CREATE TABLE IF NOT EXISTS service_requests (
    SR_NUMBER VARCHAR(50),
    SR_TYPE VARCHAR(100),
    SR_SHORT_CODE VARCHAR(10),
    CREATED_DEPARTMENT VARCHAR(100),
    OWNER_DEPARTMENT VARCHAR(100),
    STATUS VARCHAR(50),
    ORIGIN VARCHAR(50),
    CREATED_DATE TIMESTAMP,
    LAST_MODIFIED_DATE TIMESTAMP,
    CLOSED_DATE TIMESTAMP,
    STREET_ADDRESS VARCHAR(255),
    CITY VARCHAR(100),
    STATE VARCHAR(50),
    ZIP_CODE VARCHAR(20),
    STREET_NUMBER VARCHAR(20),
    STREET_DIRECTION VARCHAR(10),
    STREET_NAME VARCHAR(100),
    STREET_TYPE VARCHAR(20),
    DUPLICATE BOOLEAN,
    LEGACY_RECORD BOOLEAN,
    LEGACY_SR_NUMBER VARCHAR(50),
    PARENT_SR_NUMBER VARCHAR(50),
    COMMUNITY_AREA INT,
    WARD INT,
    ELECTRICAL_DISTRICT VARCHAR(10),
    ELECTRICITY_GRID VARCHAR(10),
    POLICE_SECTOR VARCHAR(10),
    POLICE_DISTRICT INT,
    POLICE_BEAT VARCHAR(10),
    PRECINCT INT,
    SANITATION_DIVISION_DAYS INT,
    CREATED_HOUR INT,
    CREATED_DAY_OF_WEEK INT,
    CREATED_MONTH INT,
    X_COORDINATE DOUBLE PRECISION,
    Y_COORDINATE DOUBLE PRECISION,
    LATITUDE DOUBLE PRECISION,
    LONGITUDE DOUBLE PRECISION,
    LOCATION VARCHAR(100),
    Boundaries_ZIP_Codes INT,
    Community_Areas INT,
    Zip_Codes INT,
    Census_Tracts INT,
    Wards INT,
    Boundaries_Wards_2023 INT
    );
    '''
pg_cursor.execute(create_311_table)


# community area data
create_area_table = '''
    CREATE TABLE IF NOT EXISTS communityareas (
    CommunityAreaNumber INT PRIMARY KEY,
    CommunityAreaName VARCHAR(255),
    PercentHousingCrowded DECIMAL(4, 1),
    PercentHouseholdsBelowPoverty DECIMAL(4, 1),
    PercentAged16PlusUnemployed DECIMAL(4, 1),
    PercentAged25PlusNoHighSchoolDiploma DECIMAL(4, 1),
    PercentAgedUnder18OrOver64 DECIMAL(4, 1),
    PerCapitaIncome VARCHAR,
    HardshipIndex VARCHAR
    );
    '''
pg_cursor.execute(create_area_table)


# crime data
create_crime_table = '''
    CREATE TABLE IF NOT EXISTS chicagocrimes (
    ID INT PRIMARY KEY,
    "Case Number" VARCHAR(10),
    Date TIMESTAMP,
    Block VARCHAR(50),
    IUCR VARCHAR(5),
    "Primary Type" VARCHAR(50),
    Description VARCHAR(255),
    "Location Description" VARCHAR(100),
    Arrest BOOLEAN,
    Domestic BOOLEAN,
    Beat INT,
    District INT,
    Ward INT,
    "Community Area" INT,
    "FBI Code" VARCHAR(5),
    "X Coordinate" INT,
    "Y Coordinate" INT,
    Year INT,
    "Updated On" TIMESTAMP,
    Latitude DOUBLE PRECISION,
    Longitude DOUBLE PRECISION,
    Location VARCHAR(255)
    );
    '''
pg_cursor.execute(create_crime_table)


# copy data from csv to tables
# use copy expert because column values contain ','
with open('311_Service_Requests_vacancies.csv') as f:
    pg_cursor.copy_expert('''COPY service_requests FROM STDIN WITH CSV HEADER''', f)
    

# first subquery: group neighborhoodsand sort by aggregate count (descending) of vacant property complaints
query = '''
    WITH miniquery AS (
        SELECT DISTINCT communityareas.communityareaname AS community_name, 
        service_requests.community_areas AS community_id, COUNT(*) AS num_vacancies
        FROM service_requests INNER JOIN communityareas 
        ON service_requests.community_areas = communityareas.communityareanumber 
        GROUP BY communityareas.communityareaname, service_requests.community_areas
        ORDER BY service_requests.community_areas ASC
    )
    SELECT DISTINCT miniquery.community_name, miniquery.community_id, miniquery.num_vacancies, COUNT(*) as count_violent_crime
    FROM miniquery RIGHT OUTER JOIN chicagocrimes
    ON miniquery.community_id = chicagocrimes."Community Area"
    WHERE miniquery.num_vacancies >  (select percentile_cont(0.5) within group (order by miniquery.num_vacancies DESC) as percentile_50 from miniquery)
    AND chicagocrimes."Primary Type" ILIKE ANY(ARRAY['Criminal Sexual Assault', 'Assault', 'Battery', 'Homicide', 'Robbery', 'Motor Vehicle Theft'])
    GROUP BY miniquery.community_name, miniquery.community_id, miniquery.num_vacancies
    ORDER BY COUNT(*)
    LIMIT 20;
    '''
pg_cursor.execute(query)


result_cols = [description[0] for description in pg_cursor.description]
neighborhood_crimes_df = pd.DataFrame(pg_cursor.fetchall(), columns=result_cols)

In [4]:
####################### Close the PostgreSQL cursor and connection - MUST BE RUN AFTER ANY CALLS TO POSTGRES
if pg_cursor is not None:
    pg_cursor.close()
if pg_conn is not None:
    pg_conn.close()

In [5]:
neighborhood_crimes_df

,community_name,community_id,num_vacancies,count_violent_crime
0,Edison Park,9,11271,86
1,Burnside,47,7038,137
2,Mount Greenwood,74,16218,166
3,Hegewisch,55,4590,250
4,Beverly,72,28662,285
5,McKinley Park,59,25908,344
6,Clearing,64,20859,346
7,Archer Heights,57,4896,349
8,Pullman,50,51918,380
9,Avalon Park,45,75123,490


In [31]:
# import pymongo
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
client = MongoClient()

import json
import dateutil
from dateutil import parser
import glob
import os
from os.path import isfile, join
path = os.getcwd() 

In [39]:
json_files = []  # store json file names in list

# function to load json files
def load_json(filepath):
    with open(filepath) as f:
        result = json.load(f)
    return result[0]


json_filepath = path = r'../data/jsons/*.json'
json_filepaths_list = glob.glob(json_filepath)
neigborhood_jsons = []


# get files for desired neighborhoods
community_list = neighborhood_crimes_df['community_name'].tolist()
for f in json_filepaths_list:
    f_stem = os.path.splitext(os.path.basename(f))[0]
    if f_stem in community_list: 
        print(f_stem)
        neigborhood_jsons.append(load_json(f))  # load each json file

Dunning
West Lawn
Lincoln Square
Brighton Park
Calumet Heights
Archer Heights
Burnside
Hegewisch
Ashburn
Avalon Park
Edison Park
Hyde Park
Clearing
McKinley Park
Hermosa
Kenwood
Irving Park
Pullman
Beverly
Mount Greenwood


In [40]:
client = MongoClient('localhost', 27017)  # start connection to mongodb server

db = client["chicago-neighborhoods-database"]  # create a database
neighborhoods = db["neighborhoods-collection"]  # create a collection in the database

db.neighborhoods.insert_many(neigborhood_jsons)  # bulk write all json files to collection

InsertManyResult([ObjectId('65770221965fb9180e93733a'), ObjectId('65770221965fb9180e93733b'), ObjectId('65770221965fb9180e93733c'), ObjectId('65770221965fb9180e93733d'), ObjectId('65770221965fb9180e93733e'), ObjectId('65770221965fb9180e93733f'), ObjectId('65770221965fb9180e937340'), ObjectId('65770221965fb9180e937341'), ObjectId('65770221965fb9180e937342'), ObjectId('65770221965fb9180e937343'), ObjectId('65770221965fb9180e937344'), ObjectId('65770221965fb9180e937345'), ObjectId('65770221965fb9180e937346'), ObjectId('65770221965fb9180e937347'), ObjectId('65770221965fb9180e937348'), ObjectId('65770221965fb9180e937349'), ObjectId('65770221965fb9180e93734a'), ObjectId('65770221965fb9180e93734b'), ObjectId('65770221965fb9180e93734c'), ObjectId('65770221965fb9180e93734d')], acknowledged=True)

In [41]:
test = db.neighborhoods.find_one()
print(test)

{'_id': ObjectId('65766f122a99dab466b7287d'), 'History': 'Settlement in the vicinity of Jefferson Park began in the 1830s with John Kinzie Clark and Elijah Wentworth, whose claim was near what is now the Jefferson Park Metra Station, where he operated a tavern and inn. The tiny settlement of traders, hunters, and farmers consisted of simple one and two room log cabins until Abram Gale, for whom Gale Street is named, built the first frame house in Jefferson. Jefferson Park became the hub of an independent township that was incorporated at the nearby Dickinson Tavern as Jefferson Township in 1850 until annexed by the city of Chicago in 1889. The area was once home to a significant population of Volga Germans, and one of the area\'s one time local landmarks was a local apartment building in the vicinity of the park along Higgins Avenue known by locals as "the Russian Hotel".\nJefferson Park is also home to the Northwest Chicago Historical Society which is dedicated to preserve the area\'s

In [42]:
# filter out: gang, historic
# look for: "Transportation" section tag
transportation_filter = db.neighborhoods.find({ 'Transportation' : {'$exists': True} })
# db.neighborhoods.aggregate(
#     [ "$ne": {"$match":{ "Description" : 'female'}} ]

# )